<a href="https://colab.research.google.com/github/cesarpena2529/prospect-to-mlb-player/blob/main/Capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install thefuzz[speedup] pandas

In [20]:
from google.colab import files
uploaded = files.upload()

Saving War-archeive-2006-2024 - war_daily_bat.csv to War-archeive-2006-2024 - war_daily_bat (1).csv


In [21]:
from google.colab import files
uploaded = files.upload()

Saving minor_league_combined.csv to minor_league_combined (1).csv


In [22]:
import os

# List all uploaded files
os.listdir()


['.config',
 'War-archeive-2006-2024 - war_daily_bat (1).csv',
 'minor_league_grouped_no_mex_2024.csv',
 'minor_league_combined.csv',
 'minor_league_grouped_no_mex (1).csv',
 'fuzzy_name_matches.csv',
 'War-archeive-2006-2024 - war_daily_bat.csv',
 'minor_league_combined (1).csv',
 'minor_league_grouped_fuzzy_labeled.csv',
 'sample_data']

In [23]:
import pandas as pd
from thefuzz import process

# Load datasets
minor_df = pd.read_csv("minor_league_combined.csv")
war_df = pd.read_csv("War-archeive-2006-2024 - war_daily_bat.csv")

# Normalize names
minor_df["Name"] = minor_df["Name"].astype(str).str.strip().str.lower()
war_df["name_common"] = war_df["name_common"].astype(str).str.strip().str.lower()

# Get unique names
minor_names = minor_df["Name"].unique()
war_names = war_df["name_common"].unique()

# Fuzzy match: for each minor league name, find best WAR match and score
results = []
for name in minor_names:
    best_match, score = process.extractOne(name, war_names)
    results.append((name, best_match, score))

# Create results DataFrame
matches_df = pd.DataFrame(results, columns=["Minor_League_Name", "Closest_WAR_Name", "Similarity_Score"])

# Optional: filter results where the match isn't perfect
fuzzy_only = matches_df[matches_df["Similarity_Score"] < 100].sort_values(by="Similarity_Score", ascending=False)

# Show top fuzzy matches
print(fuzzy_only.head(25))


        Minor_League_Name     Closest_WAR_Name  Similarity_Score
32    arismendy alcantara  arismendy alcántara                97
21      enrique hernandez    enrique hernández                97
86        ronny rodriguez      ronny rodríguez                97
140     teoscar hernandez    teoscar hernández                97
20     adeiny hechavarria   adeiny hechavarría                97
255       jose marmolejos      josé marmolejos                97
194      jesmuel valentin     jesmuel valentín                97
706      hernan iribarren     hernán iribarren                97
690      osvaldo martinez     osvaldo martínez                97
698      emilio bonifacio     emilio bonifácio                97
381      yadiel hernandez     yadiel hernández                97
2294      marco hernandez      marco hernández                97
716        steve tolleson      steven tolleson                97
593       luisangel acuna      luisangel acuña                97
705      gorkys hernandez

In [24]:
# Show all matches below perfect
fuzzy_only.to_csv("fuzzy_name_matches.csv", index=False)


In [25]:
import pandas as pd
from thefuzz import process

# Load files
minor_df = pd.read_csv("minor_league_combined.csv")
war_df = pd.read_csv("War-archeive-2006-2024 - war_daily_bat.csv")

# Normalize names
minor_df["Name"] = minor_df["Name"].astype(str).str.strip().str.lower()
war_df["name_common"] = war_df["name_common"].astype(str).str.strip().str.lower()

# Get unique names
minor_names = minor_df["Name"].unique()
war_names = war_df["name_common"].unique()

# Step 1: Fuzzy match each minor league name to closest WAR name
matches = []
for name in minor_names:
    match, score = process.extractOne(name, war_names)
    matches.append((name, match, score))

fuzzy_df = pd.DataFrame(matches, columns=["minor_name", "war_name", "similarity"])
fuzzy_df.to_csv("fuzzy_name_matches.csv", index=False)  # Optional audit trail

# Step 2: Get Career WAR + MLB Success (≥ 2 WAR)
career_war = war_df.groupby("name_common")["WAR"].sum().reset_index()
career_war.columns = ["war_name", "Career_WAR"]
career_war["MLB_success"] = career_war["Career_WAR"] >= 2

# Step 3: Merge fuzzy matches with WAR info
fuzzy_labeled = pd.merge(fuzzy_df, career_war, on="war_name", how="left")

# Step 4: Merge labels back into raw minor league data
minor_df = pd.merge(minor_df, fuzzy_labeled[["minor_name", "Career_WAR", "MLB_success"]],
                    left_on="Name", right_on="minor_name", how="left")

# Fill missing labels
minor_df["Career_WAR"] = minor_df["Career_WAR"].fillna(0)
minor_df["MLB_success"] = minor_df["MLB_success"].fillna(False)

# Step 5: Group by player + level (average rate stats, sum counting stats)
group_keys = ['Name', 'level']

# Customize as needed
avg_cols = ['BB%', 'K%', 'ISO', 'BABIP', 'AVG', 'OBP', 'SLG', 'OPS', 'wRC+']
sum_cols = ['G', 'PA', 'AB', 'H', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SO', 'SB', 'CS']

avg_cols = [col for col in avg_cols if col in minor_df.columns]
sum_cols = [col for col in sum_cols if col in minor_df.columns]

# Aggregate
grouped = minor_df.groupby(group_keys).agg(
    {**{col: 'mean' for col in avg_cols}, **{col: 'sum' for col in sum_cols}}
).reset_index()

# Add success labels (per player)
labels = minor_df[['Name', 'MLB_success', 'Career_WAR']].drop_duplicates(subset='Name')
grouped = pd.merge(grouped, labels, on='Name', how='left')

# Save final file
grouped.to_csv("minor_league_grouped_fuzzy_labeled.csv", index=False)
print("✅ Saved: minor_league_grouped_fuzzy_labeled.csv")


✅ Saved: minor_league_grouped_fuzzy_labeled.csv


In [26]:
from google.colab import files
files.download("minor_league_grouped_fuzzy_labeled.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>